In [4]:
# PACKAGES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import urllib.request
import urllib
import urllib.parse
import os
import requests
from datetime import datetime
# import prettytable
# import seaborn as sns
# import plotly.express as px

In [5]:
pip install wwo-hist

  Using cached wwo_hist-0.0.5-py3-none-any.whl (4.6 kB)



In [7]:
from wwo_hist import retrieve_hist_data

In [8]:
## For local use only
# import os
# os.chdir(".\YOUR_PATH")

In [45]:
capitals_list = ['Albany','Annapolis','Atlanta','Augusta','Austin','Baton_Rouge','Bismarck','Boise','Boston','Carson_City','Charleston','Cheyenne','Columbia','Columbus','Concord','Denver','Des_Moines','Dover','Frankfort','Harrisburg','Hartford','Helena','Honolulu','Indianapolis','Jackson','Jefferson_City','Juneau','Lansing','Lincoln','LittleRock','Madison','Montgomery','Montpelier','Nashville','Oklahoma_City','Olympia','Phoenix','Pierre','Providence','Raleigh','Richmond','Sacramento','Saint_Paul','Salem','Salt_Lake_City','Santa_Fe_USA','Springfield','Tallahassee','Topeka','Trenton']

states_list = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New_Hampshire','New_Jersey','New Mexico','New_York','North_Carolina','North_Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode_Island','South_Carolina','South_Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West_Virginia','Wisconsin','Wyoming']
len(capitals_list)

50

In [47]:
# Got stopped because of query limit and need to query these cities still as of 9/08
capitals_list_retry = ['Albany','Austin','Bismarck','Carson_City','Charleston','Cheyenne','Columbia','Columbus','Concord','Harrisburg','Helena','Jackson','Jefferson_City','Lincoln','Madison','Montpelier','Nashville','Oklahoma_City','Olympia','Pierre','Providence','Raleigh','Richmond','Saint_Paul','Salem','Salt_Lake_City','Santa_Fe_USA','Trenton']
len(capitals_list_retry)

27

In [52]:
# USING THE API
#########################################################
frequency = 24  # time between each data row (hours) - max of 24
start_date = '01-JAN-2015'
end_date = '31-DEC-2018'
#api_key = '831e61eaec474f5ca63202520200309'
new_api_key = 'ddd3168a425b434f828201815200309'
location_list = capitals_list_retry
hist_weather_data = retrieve_hist_data(new_api_key,
                                location_list,
                                start_date,
                                end_date,
                                frequency,
                                location_label = False,
                                export_csv = True,
                                store_df = True)



Retrieving weather data for Austin


Currently retrieving data for Austin: from 2015-01-01 to 2015-01-31


HTTPError: HTTP Error 403: Forbidden

In [13]:
# Load in .csv files and concatenate into one dataframe, then make a function for when we pull in all 5-years of data
albany = pd.read_csv('Albany.csv')
albany.head()

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2019-01-01,7,4,0.0,6.9,2,26,04:01 AM,02:42 PM,08:25 AM,...,34,58,76,13.7,1009,7,9,303,13,Albany
1,2019-01-02,0,-6,0.0,8.7,2,19,05:05 AM,03:17 PM,08:25 AM,...,14,22,57,0.0,1021,0,10,164,12,Albany
2,2019-01-03,2,-4,0.0,6.9,1,11,06:07 AM,03:55 PM,08:25 AM,...,21,52,60,0.2,1019,2,10,299,9,Albany
3,2019-01-04,2,-1,0.0,8.7,2,4,07:05 AM,04:38 PM,08:25 AM,...,13,6,79,0.0,1005,2,10,154,9,Albany
4,2019-01-05,2,0,0.0,3.4,1,0,07:59 AM,05:26 PM,08:25 AM,...,13,61,83,5.2,1004,2,8,334,7,Albany


In [39]:
df = albany.copy()
df = df.drop(['winddirDegree', 'visibility', 'pressure', 'moonrise', 'moonset', 'sunrise', 'sunset', 'DewPointC', 'HeatIndexC', 'WindChillC', 'WindGustKmph', 'sunHour', 'uvIndex', 'moon_illumination', 'tempC'], axis=1)
df.head()

,date_time,maxtempC,mintempC,totalSnow_cm,FeelsLikeC,cloudcover,humidity,precipMM,windspeedKmph,location
0,2019-01-01,7,4,0.0,1,58,76,13.7,13,Albany
1,2019-01-02,0,-6,0.0,-5,22,57,0.0,12,Albany
2,2019-01-03,2,-4,0.0,-5,52,60,0.2,9,Albany
3,2019-01-04,2,-1,0.0,-1,6,79,0.0,9,Albany
4,2019-01-05,2,0,0.0,-1,61,83,5.2,7,Albany


In [40]:
cols = list(df)
# print(len(cols))
cols.insert(0, cols.pop(cols.index('location')))
cols.insert(9, cols.pop(cols.index('date_time')))
cols.insert(3, cols.pop(cols.index('precipMM')))
cols.insert(5, cols.pop(cols.index('windspeedKmph')))
cols.insert(6, cols.pop(cols.index('humidity')))
cols.insert(7, cols.pop(cols.index('cloudcover')))
cols.insert(3, cols.pop(cols.index('FeelsLikeC')))
df = df.loc[:, cols]
df

,location,maxtempC,mintempC,FeelsLikeC,precipMM,totalSnow_cm,windspeedKmph,humidity,cloudcover,date_time
0,Albany,7,4,1,13.7,0.0,13,76,58,2019-01-01
1,Albany,0,-6,-5,0.0,0.0,12,57,22,2019-01-02
2,Albany,2,-4,-5,0.2,0.0,9,60,52,2019-01-03
3,Albany,2,-1,-1,0.0,0.0,9,79,6,2019-01-04
4,Albany,2,0,-1,5.2,0.0,7,83,61,2019-01-05
...,...,...,...,...,...,...,...,...,...,...
361,Albany,5,0,-1,0.0,0.0,10,83,51,2019-12-28
362,Albany,4,-1,0,10.8,0.0,4,85,48,2019-12-29
363,Albany,5,1,1,40.1,0.0,8,93,97,2019-12-30
364,Albany,4,0,0,2.1,0.3,6,90,91,2019-12-31


In [ ]:
def clean(DataFrame):
    X = DataFrame.copy()
    X = X.drop(['winddirDegree', 'visibility', 'pressure', 'moonrise',                           'moonset', 'sunrise', 'sunset', 'DewPointC', 'HeatIndexC',                       'WindChillC', 'WindGustKmph', 'sunHour', 'uvIndex',                              'moon_illumination', 'tempC'], axis=1)

    cols = list(X)
    # print(len(cols))
    cols.insert(0, cols.pop(cols.index('location')))
    cols.insert(9, cols.pop(cols.index('date_time')))
    cols.insert(3, cols.pop(cols.index('precipMM')))
    cols.insert(5, cols.pop(cols.index('windspeedKmph')))
    cols.insert(6, cols.pop(cols.index('humidity')))
    cols.insert(7, cols.pop(cols.index('cloudcover')))
    cols.insert(3, cols.pop(cols.index('FeelsLikeC')))
    X = X.loc[:, cols]

    return X